In [1]:
import pandas as pd 
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import math
import os
import re 

In [2]:
df_1 = pd.read_csv(r'C:\Users\super\Desktop\MyJPY\Datas\Fund\基金经理档案.csv')
df_1.head()

,文本,页面网址
0,\n \n 基金经理：艾...,http://fund.eastmoney.com/manager/30634044.html
1,\n \n 基金经理：艾...,http://fund.eastmoney.com/manager/30277862.html
2,\n \n 基金经理：安...,http://fund.eastmoney.com/manager/30074174.html
3,\n \n 基金经理：彬...,http://fund.eastmoney.com/manager/30648059.html
4,\n \n 基金经理：包...,http://fund.eastmoney.com/manager/30659779.html


In [4]:
df_2 = pd.read_csv(r'C:\Users\super\Desktop\MyJPY\Datas\Fund\基金经理列表.csv')
df_2.head()

,姓名_链接,姓名,所属公司,现任基金,累计从业时间,现任基金资产总规模,现任基金最佳回报
0,http://fund.eastmoney.com/manager/30634044.html,艾定飞,华商基金,共2只:华商电子行业量化华商计算机行业量,2年又13天,7.34亿元,64.68%
1,http://fund.eastmoney.com/manager/30277862.html,艾小军,国泰基金,共23只:国泰黄金ETF联国泰黄金ETF联更多>,6年又331天,719.23亿元,146.40%
2,http://fund.eastmoney.com/manager/30074174.html,安昀,长信基金,共4只:长信内需成长混合长信消费升级混合更多>,6年又231天,38.49亿元,139.13%
3,http://fund.eastmoney.com/manager/30648059.html,彬彬,人保资产,共6只:人保量化混合A人保量化混合C更多>,1年又328天,5.24亿元,49.40%
4,http://fund.eastmoney.com/manager/30659779.html,包兵华,鹏华基金,共4只:鹏华研究驱动混合鹏华研究智选混合更多>,1年又233天,147.28亿元,84.37%


In [5]:
df_3 = pd.read_csv(r'C:\Users\super\Desktop\MyJPY\Datas\Fund\基金净值3(24).csv')
df_3.tail()

,基金标题,净值日期,单位净值,累计净值,日增长率,日下跌率,申购状态,赎回状态
19995,长信双利优选混合A(519991)基金历史净值 _ 基金档案 _ 天天基金网,2016-07-18,1.2830\n \n ...,1.919,NaN,-1.69%,开放申购,开放赎回
19996,长信双利优选混合A(519991)基金历史净值 _ 基金档案 _ 天天基金网,2016-07-27,1.2570\n \n ...,1.893,NaN,-2.93%,开放申购,开放赎回
19997,长信双利优选混合A(519991)基金历史净值 _ 基金档案 _ 天天基金网,2016-07-20,1.2860\n \n ...,1.922,NaN,NaN,开放申购,开放赎回
19998,安信价值精选股票(000577)基金历史净值 _ 基金档案 _ 天天基金网,2019-07-19,2.7730\n \n ...,2.773,1.58%,NaN,限制大额申购,开放赎回
19999,安信价值精选股票(000577)基金历史净值 _ 基金档案 _ 天天基金网,2019-07-18,2.7300\n \n ...,2.730,NaN,-0.73%,限制大额申购,开放赎回


基金走势对比
筛选了5只自己比较关注的基金，如需查看其他基金的数据，Fork之后更新下方cell中的CODE列表；
成立以来

In [7]:
# 用于展示的基金代码
# 如需查看其他基金的数据，替换此list内容
CODE = ['320007', '161725', '003095', '004698', '110022']

def found_code(str_):
    pattern = re.compile('.*([0-9]{6}?).*基金历史净值')
    m = pattern.match(str_)
    code = m.group(1)
    return code


df_3 = pd.DataFrame()
for f_path, dir_name, f_names in os.walk(r'C:\Users\super\Desktop\MyJPY\Datas\Fund'):
    for fn in f_names:
        if fn.startswith('基金净值'):
            try:
                df_temp = pd.read_csv(os.path.join(f_path, fn))
                df_temp['基金代码'] = df_temp['基金标题'].apply(found_code)
                df_temp = df_temp[df_temp['基金代码'].isin(CODE)]
                if df_temp.empty:
                    pass
                else:
                    df_3 = df_3.append(df_temp, ignore_index=True, sort=False)
                    # print(os.path.join(f_path, fn), 'DONE')
            except:
                print(os.path.join(f_path, fn), 'ERROR')
df_3['净值日期']=pd.to_datetime(df_3['净值日期'])
df_3.sort_values(by=['净值日期', '基金代码'], inplace=True)

def found_name(str_):
    pattern = re.compile('(.*?)\(.*')
    m = pattern.match(str_)
    code = m.group(1)
    return code

df_3['基金名称'] = df_3['基金标题'].apply(found_name)

In [8]:
max_date = df_3['净值日期'].max()
min_date = df_3['净值日期'].min()
data_x = df_3['净值日期'][df_3['净值日期'].isin(pd.date_range(start=min_date, end=max_date))].unique()
data_x = [pd.to_datetime(x).strftime('%Y-%m-%d') for x in data_x]
days = len(data_x)
 
bg_color_js = """
new echarts.graphic.RadialGradient(0.3, 0.3, 0.8, [{
        offset: 0,
        color: '#f7f8fa'
    }, {
        offset: 1,
        color: '#cdd0d5'
    }])"""
    
line = Line(init_opts=opts.InitOpts(theme='light', bg_color=JsCode(bg_color_js), width='1200px', height='600px'))
line.add_xaxis(data_x)
 
for code in CODE:
    df_3_temp = df_3[df_3['基金代码']==code]
    name = df_3_temp['基金名称'].max()
    money = 10000
    data_y = []
    for _, row in df_3_temp.iterrows():
        diff_rate = row['日增长率'] if str(row['日增长率']).endswith('%') else row['日下跌率']
        diff_rate = float(diff_rate.replace('%', '') if str(diff_rate).endswith('%') else 0)
        money = money + money * (diff_rate/100)
        data_y.append(money/100-100)
 
    
    if len(data_y) != days:
        for _ in range(days-len(data_y)):
            data_y.insert(0, None)
    line.add_yaxis(name,
                    data_y,
                    is_symbol_show=True,
                    symbol='circle',
                    symbol_size=1,
                    is_smooth=True,
                    linestyle_opts={'shadowBlur': 5,
                                    'shadowColor': 'rgba(0, 0, 0, 0.5)',
                                    'shadowOffsetY': 10,
                                    'shadowOffsetX': 10,
                                    'width':2,
                                    'curve':0.1,
                                    # 'color': '#000000'
                                    })
# line.add_yaxis('Yeezy',
#                data_y,
#                is_symbol_show=False,
#                linestyle_opts={'shadowBlur': 5,
#                                 'shadowColor': 'rgba(0, 0, 0, 0.5)',
#                                 'shadowOffsetY': 10,
#                                 'shadowOffsetX': 10,
#                                 'width':5,
#                                 'curve':0.5,
#                                 'color': '#3CB371'
#                                 })
line.set_series_opts(label_opts=opts.LabelOpts(is_show=False),)
line.set_global_opts(yaxis_opts=opts.AxisOpts(name='涨跌/%', type_="value", is_scale=True,
                                            splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),
                                            axislabel_opts=opts.LabelOpts(formatter='{value}%')),
                     xaxis_opts=opts.AxisOpts(name='日期',#type_="time",
                                            splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                     legend_opts=opts.LegendOpts(is_show=True, pos_left=200, pos_top=100, orient='vertical', item_gap=20,
                                                 textstyle_opts=opts.TextStyleOpts(color='#fff', font_size=13, font_weight='bold')),
                     title_opts=opts.TitleOpts(title="业绩走势对比[成立以来]", pos_left='center',
                                            title_textstyle_opts=opts.TextStyleOpts(color='#08A05C', font_size=20, font_weight='bold'),
                                            subtitle='数据来源：天天基金', 
                                            ),
                     graphic_opts=[
                         opts.GraphicGroup(
                            graphic_item=opts.GraphicItem(id_='3',left="180px", top="80px"),
                            children=[
                                    opts.GraphicRect(
                                                            graphic_item=opts.GraphicItem(
                                                                left="180px", top="80px", z=-1
                                                            ),
                                                            graphic_shape_opts=opts.GraphicShapeOpts(
                                                                width=220, height=200
                                                            ),
                                                            graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                                                # 颜色配置，这里设置为黑色，透明度为0.5
                                                                fill="rgba(0,0,0,0.3)"
                                                            ),
                                                        )
                            ],
                        ),]
                     
                     )
line.render_notebook()

In [9]:
max_date = df_3['净值日期'].max()
min_date = pd.datetime(2017,1,1)
data_x = df_3['净值日期'][df_3['净值日期'].isin(pd.date_range(start=min_date, end=max_date))].unique()
data_x = [pd.to_datetime(x).strftime('%Y-%m-%d') for x in data_x]
days = len(data_x)
 
bg_color_js = """
new echarts.graphic.RadialGradient(0.3, 0.3, 0.8, [{
        offset: 0,
        color: '#f7f8fa'
    }, {
        offset: 1,
        color: '#cdd0d5'
    }])"""
    
line = Line(init_opts=opts.InitOpts(theme='light', bg_color=JsCode(bg_color_js), width='1200px', height='600px'))
line.add_xaxis(data_x)
 
for code in CODE:
    df_3_temp = df_3[df_3['净值日期'].isin(data_x) & (df_3['基金代码']==code)]
    name = df_3_temp['基金名称'].max()
    money = 10000
    data_y = []
    for _, row in df_3_temp.iterrows():
        diff_rate = row['日增长率'] if str(row['日增长率']).endswith('%') else row['日下跌率']
        diff_rate = float(diff_rate.replace('%', '') if str(diff_rate).endswith('%') else 0)
        money = money + money * (diff_rate/100)
        data_y.append(money/100-100)
 
    
    if len(data_y) != days:
        for _ in range(days-len(data_y)):
            data_y.insert(0, None)
    line.add_yaxis(name,
                    data_y,
                    is_symbol_show=True,
                    symbol='circle',
                    symbol_size=1,
                    is_smooth=True,
                    linestyle_opts={'shadowBlur': 5,
                                    'shadowColor': 'rgba(0, 0, 0, 0.5)',
                                    'shadowOffsetY': 10,
                                    'shadowOffsetX': 10,
                                    'width':2,
                                    'curve':0.1,
                                    # 'color': '#000000'
                                    })
# line.add_yaxis('Yeezy',
#                data_y,
#                is_symbol_show=False,
#                linestyle_opts={'shadowBlur': 5,
#                                 'shadowColor': 'rgba(0, 0, 0, 0.5)',
#                                 'shadowOffsetY': 10,
#                                 'shadowOffsetX': 10,
#                                 'width':5,
#                                 'curve':0.5,
#                                 'color': '#3CB371'
#                                 })
line.set_series_opts(label_opts=opts.LabelOpts(is_show=False),)
line.set_global_opts(yaxis_opts=opts.AxisOpts(name='涨跌/%', type_="value", is_scale=True,
                                            splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),
                                            axislabel_opts=opts.LabelOpts(formatter='{value}%')),
                     xaxis_opts=opts.AxisOpts(name='日期',#type_="time",
                                            splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                     legend_opts=opts.LegendOpts(is_show=True, pos_left=200, pos_top=100, orient='vertical', item_gap=20,
                                                 textstyle_opts=opts.TextStyleOpts(color='#fff', font_size=13, font_weight='bold')),
                     title_opts=opts.TitleOpts(title="业绩走势对比[近三年]", pos_left='center',
                                            title_textstyle_opts=opts.TextStyleOpts(color='#08A05C', font_size=20, font_weight='bold'),
                                            subtitle='数据来源：天天基金', 
                                            ),
                     graphic_opts=[
                         opts.GraphicGroup(
                            graphic_item=opts.GraphicItem(id_='3',left="180px", top="80px"),
                            children=[
                                    opts.GraphicRect(
                                                            graphic_item=opts.GraphicItem(
                                                                left="180px", top="80px", z=-1
                                                            ),
                                                            graphic_shape_opts=opts.GraphicShapeOpts(
                                                                width=220, height=200
                                                            ),
                                                            graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                                                # 颜色配置，这里设置为黑色，透明度为0.5
                                                                fill="rgba(0,0,0,0.3)"
                                                            ),
                                                        )
                            ],
                        ),]
                     
                     )
line.render_notebook()

c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


1W本金一年能赚多少钱
诺安成长混合
如果从2020-1-1投入1W买诺安，到7月中旬达到最大最大收益，1W本金可获得8000+收益，虽然后期一路走跌，但到12月份到时候还是能有4000左右的收益；

In [10]:
max_date = df_3['净值日期'].max()
min_date = pd.datetime(2020,1,1)
data_x = df_3['净值日期'][df_3['净值日期'].isin(pd.date_range(start=min_date, end=max_date))].unique()
data_x = [pd.to_datetime(x).strftime('%Y-%m-%d') for x in data_x]
days = len(data_x)
 
df_3_temp = df_3[df_3['净值日期'].isin(data_x) & (df_3['基金代码']==CODE[0])]
name = df_3_temp['基金名称'].max()
data_y_day, data_y_total = [], []
money = 10000
for _, row in df_3_temp.iterrows():
    diff_rate = row['日增长率'] if str(row['日增长率']).endswith('%') else row['日下跌率']
    diff_rate = float(diff_rate.replace('%', '') if str(diff_rate).endswith('%') else 0)
    money = money + money * (diff_rate/100)
    money_diff = money * (diff_rate/100)
    data_y_day.append(int(money_diff))
    data_y_total.append(money)
 
    if len(data_y) != days:
        for _ in range(days-len(data_y)):
            data_y.insert(0, None)
 
 
 
bar = Bar(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
bar.add_xaxis(data_x)
bar.add_yaxis(name,
              data_y_day, stack='1',
              itemstyle_opts={"normal": {
                  'shadowBlur': 10,
                  'shadowColor': 'rgba(0, 0, 0, 0.5)',
                  'shadowOffsetX': 10,
                  'shadowOffsetY': 10,
                  # 'color':'red',
                  'borderColor': 'rgb(220,220,220)',
                  'borderWidth': 2}
              },
              gap='50%',
              # bar_width=20,
              tooltip_opts=opts.TooltipOpts(is_show=True, formatter='{b}<br>当日收益：{c}元'),
              label_opts=opts.LabelOpts(is_show=False))
 
bar.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      is_piecewise=True,
                                                      # dimension=0,
                                                      pieces=[{"max": 0, "color": 'green'},
                                                              {"min": 1,
                                                                  "color": 'red'},
                                                              ]),
                    yaxis_opts=opts.AxisOpts(name='日收益/元', type_="value", is_scale=True,
                                             splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),),
                    xaxis_opts=opts.AxisOpts(name='日期',  # type_="time",
                                             splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                    legend_opts=opts.LegendOpts(is_show=False),
                    )
line = Line(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
line.add_xaxis(data_x)
line.add_yaxis(name,
               data_y_total,
               is_symbol_show=True,
               symbol='circle',
               symbol_size=1,
               is_smooth=True,
               linestyle_opts={'shadowBlur': 5,
                               'shadowColor': 'rgba(0, 0, 0, 0.5)',
                               'shadowOffsetY': 10,
                               'shadowOffsetX': 10,
                               'width': 2,
                               'curve': 0.1,
                               # 'color': '#000000'
                               },
               tooltip_opts=opts.TooltipOpts(is_show=False),
               label_opts=opts.LabelOpts(is_show=False))
 
line.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                       is_piecewise=True,
                                                       # dimension=0,
                                                       pieces=[{"max": 0, "color": 'green'},
                                                               {"min": 1,
                                                                "color": 'red'},
                                                               ]),
                     yaxis_opts=opts.AxisOpts(name='金额/元', type_="value", is_scale=True,
                                              splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),),
                     xaxis_opts=opts.AxisOpts(name='日期',
                                              splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                     legend_opts=opts.LegendOpts(is_show=False),
                     title_opts=opts.TitleOpts(title="1W元本金日收益趋势", pos_left='center',
                                               title_textstyle_opts=opts.TextStyleOpts(
                                                   color='#08A05C', font_size=20, font_weight='bold'),
                                               subtitle='产品：{}'.format(name),
                                               ),
)
line.set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(type_="min", name="最小值"),
                opts.MarkLineItem(type_="max", name="最大值"),
            ]
        ),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(name="收益", coord=[data_x[-1], data_y_total[-1]], value=int(data_y_total[-1]), symbol_size=[80,50])]
        )
    )
grid = Grid(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
 
grid.add(line, is_control_axis_index=False, grid_opts=opts.GridOpts(
    pos_left='5%', pos_right='5%', pos_top='10%', pos_bottom='50%'))
grid.add(bar, is_control_axis_index=False, grid_opts=opts.GridOpts(
    pos_left='5%', pos_right='5%', pos_top='60%', pos_bottom='5%'))
 
grid.render_notebook()

c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


招商中证白酒指数分级
2020-1-1开始1W建仓的话，一年翻倍！（数据集缺少了最近一个月的数据，近一个月白酒又是30%的收益！）

In [11]:
max_date = df_3['净值日期'].max()
min_date = pd.datetime(2020,1,1)
data_x = df_3['净值日期'][df_3['净值日期'].isin(pd.date_range(start=min_date, end=max_date))].unique()
data_x = [pd.to_datetime(x).strftime('%Y-%m-%d') for x in data_x]
days = len(data_x)
 
df_3_temp = df_3[df_3['净值日期'].isin(data_x) & (df_3['基金代码']==CODE[1])]
name = df_3_temp['基金名称'].max()
data_y_day, data_y_total = [], []
money = 10000
for _, row in df_3_temp.iterrows():
    diff_rate = row['日增长率'] if str(row['日增长率']).endswith('%') else row['日下跌率']
    diff_rate = float(diff_rate.replace('%', '') if str(diff_rate).endswith('%') else 0)
    money = money + money * (diff_rate/100)
    money_diff = money * (diff_rate/100)
    data_y_day.append(money_diff)
    data_y_total.append(money)
 
    if len(data_y) != days:
        for _ in range(days-len(data_y)):
            data_y.insert(0, None)
 
 
 
bar = Bar(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
bar.add_xaxis(data_x)
bar.add_yaxis(name,
              data_y_day, stack='1',
              itemstyle_opts={"normal": {
                  'shadowBlur': 10,
                  'shadowColor': 'rgba(0, 0, 0, 0.5)',
                  'shadowOffsetX': 10,
                  'shadowOffsetY': 10,
                  # 'color':'red',
                  'borderColor': 'rgb(220,220,220)',
                  'borderWidth': 2}
              },
              gap='50%',
              # bar_width=20,
              tooltip_opts=opts.TooltipOpts(is_show=True, formatter='{b}<br>当日收益：{c}元'),
              label_opts=opts.LabelOpts(is_show=False))
 
bar.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      is_piecewise=True,
                                                      # dimension=0,
                                                      pieces=[{"max": 0, "color": 'green'},
                                                              {"min": 1,
                                                                  "color": 'red'},
                                                              ]),
                    yaxis_opts=opts.AxisOpts(name='日收益/元', type_="value", is_scale=True,
                                             splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),),
                    xaxis_opts=opts.AxisOpts(name='日期',  # type_="time",
                                             splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                    legend_opts=opts.LegendOpts(is_show=False),
                    )
line = Line(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
line.add_xaxis(data_x)
line.add_yaxis(name,
               data_y_total,
               is_symbol_show=True,
               symbol='circle',
               symbol_size=1,
               is_smooth=True,
               linestyle_opts={'shadowBlur': 5,
                               'shadowColor': 'rgba(0, 0, 0, 0.5)',
                               'shadowOffsetY': 10,
                               'shadowOffsetX': 10,
                               'width': 2,
                               'curve': 0.1,
                               # 'color': '#000000'
                               },
               tooltip_opts=opts.TooltipOpts(is_show=False),
               label_opts=opts.LabelOpts(is_show=False))
 
line.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                       is_piecewise=True,
                                                       # dimension=0,
                                                       pieces=[{"max": 0, "color": 'green'},
                                                               {"min": 1,
                                                                "color": 'red'},
                                                               ]),
                     yaxis_opts=opts.AxisOpts(name='金额/元', type_="value", is_scale=True,
                                              splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),),
                     xaxis_opts=opts.AxisOpts(name='日期',
                                              splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                     legend_opts=opts.LegendOpts(is_show=False),
                     title_opts=opts.TitleOpts(title="1W元本金日收益趋势", pos_left='center',
                                               title_textstyle_opts=opts.TextStyleOpts(
                                                   color='#08A05C', font_size=20, font_weight='bold'),
                                               subtitle='产品：{}'.format(name),
                                               ),
)
line.set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(type_="min", name="最小值"),
                opts.MarkLineItem(type_="max", name="最大值"),
            ]
        ),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(name="收益", coord=[data_x[-1], data_y_total[-1]], value=int(data_y_total[-1]), symbol_size=[80,50])]
        )
    )
grid = Grid(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
 
grid.add(line, is_control_axis_index=False, grid_opts=opts.GridOpts(
    pos_left='5%', pos_right='5%', pos_top='10%', pos_bottom='50%'))
grid.add(bar, is_control_axis_index=False, grid_opts=opts.GridOpts(
    pos_left='5%', pos_right='5%', pos_top='60%', pos_bottom='5%'))
 
grid.render_notebook()

c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


中欧医疗健康混合A
医疗8月份收益达到顶峰，之后一路涨涨跌跌，到目前的话勉强回到了7月份的最高点～

In [12]:
max_date = df_3['净值日期'].max()
min_date = pd.datetime(2020,1,8)
data_x = df_3['净值日期'][df_3['净值日期'].isin(pd.date_range(start=min_date, end=max_date))].unique()
data_x = [pd.to_datetime(x).strftime('%Y-%m-%d') for x in data_x]
days = len(data_x)
 
df_3_temp = df_3[df_3['净值日期'].isin(data_x) & (df_3['基金代码']==CODE[2])]
name = df_3_temp['基金名称'].max()
data_y_day, data_y_total = [], []
money = 10000
for _, row in df_3_temp.iterrows():
    diff_rate = row['日增长率'] if str(row['日增长率']).endswith('%') else row['日下跌率']
    diff_rate = float(diff_rate.replace('%', '') if str(diff_rate).endswith('%') else 0)
    money = money + money * (diff_rate/100)
    money_diff = money * (diff_rate/100)
    data_y_day.append(money_diff)
    data_y_total.append(money)
 
    if len(data_y) != days:
        for _ in range(days-len(data_y)):
            data_y.insert(0, None)
 
 
 
bar = Bar(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
bar.add_xaxis(data_x)
bar.add_yaxis(name,
              data_y_day, stack='1',
              itemstyle_opts={"normal": {
                  'shadowBlur': 10,
                  'shadowColor': 'rgba(0, 0, 0, 0.5)',
                  'shadowOffsetX': 10,
                  'shadowOffsetY': 10,
                  # 'color':'red',
                  'borderColor': 'rgb(220,220,220)',
                  'borderWidth': 2}
              },
              gap='50%',
              # bar_width=20,
              tooltip_opts=opts.TooltipOpts(is_show=True, formatter='{b}<br>当日收益：{c}元'),
              label_opts=opts.LabelOpts(is_show=False))
 
bar.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      is_piecewise=True,
                                                      # dimension=0,
                                                      pieces=[{"max": 0, "color": 'green'},
                                                              {"min": 1,
                                                                  "color": 'red'},
                                                              ]),
                    yaxis_opts=opts.AxisOpts(name='日收益/元', type_="value", is_scale=True,
                                             splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),),
                    xaxis_opts=opts.AxisOpts(name='日期',  # type_="time",
                                             splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                    legend_opts=opts.LegendOpts(is_show=False),
                    )
line = Line(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
line.add_xaxis(data_x)
line.add_yaxis(name,
               data_y_total,
               is_symbol_show=True,
               symbol='circle',
               symbol_size=1,
               is_smooth=True,
               linestyle_opts={'shadowBlur': 5,
                               'shadowColor': 'rgba(0, 0, 0, 0.5)',
                               'shadowOffsetY': 10,
                               'shadowOffsetX': 10,
                               'width': 2,
                               'curve': 0.1,
                               # 'color': '#000000'
                               },
               tooltip_opts=opts.TooltipOpts(is_show=False),
               label_opts=opts.LabelOpts(is_show=False))
 
line.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                       is_piecewise=True,
                                                       # dimension=0,
                                                       pieces=[{"max": 0, "color": 'green'},
                                                               {"min": 1,
                                                                "color": 'red'},
                                                               ]),
                     yaxis_opts=opts.AxisOpts(name='金额/元', type_="value", is_scale=True,
                                              splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),),
                     xaxis_opts=opts.AxisOpts(name='日期',
                                              splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                     legend_opts=opts.LegendOpts(is_show=False),
                     title_opts=opts.TitleOpts(title="1W元本金日收益趋势", pos_left='center',
                                               title_textstyle_opts=opts.TextStyleOpts(
                                                   color='#08A05C', font_size=20, font_weight='bold'),
                                               subtitle='产品：{}'.format(name),
                                               ),
)
line.set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(type_="min", name="最小值"),
                opts.MarkLineItem(type_="max", name="最大值"),
            ]
        ),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(name="收益", coord=[data_x[-1], data_y_total[-1]], value=int(data_y_total[-1]), symbol_size=[80,50])]
        )
    )
grid = Grid(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
 
grid.add(line, is_control_axis_index=False, grid_opts=opts.GridOpts(
    pos_left='5%', pos_right='5%', pos_top='10%', pos_bottom='50%'))
grid.add(bar, is_control_axis_index=False, grid_opts=opts.GridOpts(
    pos_left='5%', pos_right='5%', pos_top='60%', pos_bottom='5%'))
 
grid.render_notebook()

c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


博时军工主题股票
军工也是近期走势很猛的一只基金，近一个月30%左右的收益，如果从年初开始建仓，到目前也是可以实现翻倍的～

In [13]:
max_date = df_3['净值日期'].max()
min_date = pd.datetime(2020,1,8)
data_x = df_3['净值日期'][df_3['净值日期'].isin(pd.date_range(start=min_date, end=max_date))].unique()
data_x = [pd.to_datetime(x).strftime('%Y-%m-%d') for x in data_x]
days = len(data_x)
 
df_3_temp = df_3[df_3['净值日期'].isin(data_x) & (df_3['基金代码']==CODE[3])]
name = df_3_temp['基金名称'].max()
data_y_day, data_y_total = [], []
money = 10000
for _, row in df_3_temp.iterrows():
    diff_rate = row['日增长率'] if str(row['日增长率']).endswith('%') else row['日下跌率']
    diff_rate = float(diff_rate.replace('%', '') if str(diff_rate).endswith('%') else 0)
    money = money + money * (diff_rate/100)
    money_diff = money * (diff_rate/100)
    data_y_day.append(money_diff)
    data_y_total.append(money)
 
    if len(data_y) != days:
        for _ in range(days-len(data_y)):
            data_y.insert(0, None)
 
 
 
bar = Bar(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
bar.add_xaxis(data_x)
bar.add_yaxis(name,
              data_y_day, stack='1',
              itemstyle_opts={"normal": {
                  'shadowBlur': 10,
                  'shadowColor': 'rgba(0, 0, 0, 0.5)',
                  'shadowOffsetX': 10,
                  'shadowOffsetY': 10,
                  # 'color':'red',
                  'borderColor': 'rgb(220,220,220)',
                  'borderWidth': 2}
              },
              gap='50%',
              # bar_width=20,
              tooltip_opts=opts.TooltipOpts(is_show=True, formatter='{b}<br>当日收益：{c}元'),
              label_opts=opts.LabelOpts(is_show=False))
 
bar.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      is_piecewise=True,
                                                      # dimension=0,
                                                      pieces=[{"max": 0, "color": 'green'},
                                                              {"min": 1,
                                                                  "color": 'red'},
                                                              ]),
                    yaxis_opts=opts.AxisOpts(name='日收益/元', type_="value", is_scale=True,
                                             splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),),
                    xaxis_opts=opts.AxisOpts(name='日期',  # type_="time",
                                             splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                    legend_opts=opts.LegendOpts(is_show=False),
                    )
line = Line(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
line.add_xaxis(data_x)
line.add_yaxis(name,
               data_y_total,
               is_symbol_show=True,
               symbol='circle',
               symbol_size=1,
               is_smooth=True,
               linestyle_opts={'shadowBlur': 5,
                               'shadowColor': 'rgba(0, 0, 0, 0.5)',
                               'shadowOffsetY': 10,
                               'shadowOffsetX': 10,
                               'width': 2,
                               'curve': 0.1,
                               # 'color': '#000000'
                               },
               tooltip_opts=opts.TooltipOpts(is_show=False),
               label_opts=opts.LabelOpts(is_show=False))
 
line.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                       is_piecewise=True,
                                                       # dimension=0,
                                                       pieces=[{"max": 0, "color": 'green'},
                                                               {"min": 1,
                                                                "color": 'red'},
                                                               ]),
                     yaxis_opts=opts.AxisOpts(name='金额/元', type_="value", is_scale=True,
                                              splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),),
                     xaxis_opts=opts.AxisOpts(name='日期',
                                              splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                     legend_opts=opts.LegendOpts(is_show=False),
                     title_opts=opts.TitleOpts(title="1W元本金日收益趋势", pos_left='center',
                                               title_textstyle_opts=opts.TextStyleOpts(
                                                   color='#08A05C', font_size=20, font_weight='bold'),
                                               subtitle='产品：{}'.format(name),
                                               ),
)
line.set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(type_="min", name="最小值"),
                opts.MarkLineItem(type_="max", name="最大值"),
            ]
        ),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(name="收益", coord=[data_x[-1], data_y_total[-1]], value=int(data_y_total[-1]), symbol_size=[80,50])]
        )
    )
grid = Grid(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
 
grid.add(line, is_control_axis_index=False, grid_opts=opts.GridOpts(
    pos_left='5%', pos_right='5%', pos_top='10%', pos_bottom='50%'))
grid.add(bar, is_control_axis_index=False, grid_opts=opts.GridOpts(
    pos_left='5%', pos_right='5%', pos_top='60%', pos_bottom='5%'))
 
grid.render_notebook()

c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


易方达消费行业股票
消费赛道的明基金，虽然不是很“猛”，但是稳，而且这也是一只经历过历史考验的基金，自肖楠2012年接手以来，

In [14]:
max_date = df_3['净值日期'].max()
min_date = pd.datetime(2020,1,8)
data_x = df_3['净值日期'][df_3['净值日期'].isin(pd.date_range(start=min_date, end=max_date))].unique()
data_x = [pd.to_datetime(x).strftime('%Y-%m-%d') for x in data_x]
days = len(data_x)
 
df_3_temp = df_3[df_3['净值日期'].isin(data_x) & (df_3['基金代码']==CODE[4])]
name = df_3_temp['基金名称'].max()
data_y_day, data_y_total = [], []
money = 10000
for _, row in df_3_temp.iterrows():
    diff_rate = row['日增长率'] if str(row['日增长率']).endswith('%') else row['日下跌率']
    diff_rate = float(diff_rate.replace('%', '') if str(diff_rate).endswith('%') else 0)
    money = money + money * (diff_rate/100)
    money_diff = money * (diff_rate/100)
    data_y_day.append(money_diff)
    data_y_total.append(money)
 
    if len(data_y) != days:
        for _ in range(days-len(data_y)):
            data_y.insert(0, None)
 
 
 
bar = Bar(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
bar.add_xaxis(data_x)
bar.add_yaxis(name,
              data_y_day, stack='1',
              itemstyle_opts={"normal": {
                  'shadowBlur': 10,
                  'shadowColor': 'rgba(0, 0, 0, 0.5)',
                  'shadowOffsetX': 10,
                  'shadowOffsetY': 10,
                  # 'color':'red',
                  'borderColor': 'rgb(220,220,220)',
                  'borderWidth': 2}
              },
              gap='50%',
              # bar_width=20,
              tooltip_opts=opts.TooltipOpts(is_show=True, formatter='{b}<br>当日收益：{c}元'),
              label_opts=opts.LabelOpts(is_show=False))
 
bar.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      is_piecewise=True,
                                                      # dimension=0,
                                                      pieces=[{"max": 0, "color": 'green'},
                                                              {"min": 1,
                                                                  "color": 'red'},
                                                              ]),
                    yaxis_opts=opts.AxisOpts(name='日收益/元', type_="value", is_scale=True,
                                             splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),),
                    xaxis_opts=opts.AxisOpts(name='日期',  # type_="time",
                                             splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                    legend_opts=opts.LegendOpts(is_show=False),
                    )
line = Line(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
line.add_xaxis(data_x)
line.add_yaxis(name,
               data_y_total,
               is_symbol_show=True,
               symbol='circle',
               symbol_size=1,
               is_smooth=True,
               linestyle_opts={'shadowBlur': 5,
                               'shadowColor': 'rgba(0, 0, 0, 0.5)',
                               'shadowOffsetY': 10,
                               'shadowOffsetX': 10,
                               'width': 2,
                               'curve': 0.1,
                               # 'color': '#000000'
                               },
               tooltip_opts=opts.TooltipOpts(is_show=False),
               label_opts=opts.LabelOpts(is_show=False))
 
line.set_global_opts(visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                       is_piecewise=True,
                                                       # dimension=0,
                                                       pieces=[{"max": 0, "color": 'green'},
                                                               {"min": 1,
                                                                "color": 'red'},
                                                               ]),
                     yaxis_opts=opts.AxisOpts(name='金额/元', type_="value", is_scale=True,
                                              splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed')),),
                     xaxis_opts=opts.AxisOpts(name='日期',
                                              splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(type_='dashed'))),
                     legend_opts=opts.LegendOpts(is_show=False),
                     title_opts=opts.TitleOpts(title="1W元本金日收益趋势", pos_left='center',
                                               title_textstyle_opts=opts.TextStyleOpts(
                                                   color='#08A05C', font_size=20, font_weight='bold'),
                                               subtitle='产品：{}'.format(name),
                                               ),
)
line.set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(type_="min", name="最小值"),
                opts.MarkLineItem(type_="max", name="最大值"),
            ]
        ),
        markpoint_opts=opts.MarkPointOpts(
            data=[opts.MarkPointItem(name="收益", coord=[data_x[-1], data_y_total[-1]], value=int(data_y_total[-1]), symbol_size=[80,50])]
        )
    )
grid = Grid(
    init_opts=opts.InitOpts(
        theme='white',
        bg_color=JsCode(bg_color_js),
        width='1200px',
        height='800px'))
 
grid.add(line, is_control_axis_index=False, grid_opts=opts.GridOpts(
    pos_left='5%', pos_right='5%', pos_top='10%', pos_bottom='50%'))
grid.add(bar, is_control_axis_index=False, grid_opts=opts.GridOpts(
    pos_left='5%', pos_right='5%', pos_top='60%', pos_bottom='5%'))
 
grid.render_notebook()

c:\users\super\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


基金经理
从业经验分布

In [15]:
import re

def parase_work_years(str_):
    pattern = re.compile('([0-9]*?)[年又]*?([0-9]+?)天')
    m = pattern.match(str_)
    year, day = m.groups()
    if year:
        year = int(year) + int(day) / 365
    else:
        year = int(day) / 365
    return year

df_2['累计从业时间(年)'] = df_2['累计从业时间'].apply(parase_work_years)
df_2['累计从业时间(离散)'] = pd.cut(df_2['累计从业时间(年)'], bins=[0, 1, 3, 5, 10, float('inf')],
                                 labels=["1年以下", "1-3年", "3-5年", '5-10年', '10年以上'])
                                 
data = dict(df_2['累计从业时间(离散)'].value_counts())


pie = Pie(init_opts=opts.InitOpts(theme='light',
                                  width='1000px',
                                  height='600px'))
pie.add("",
        [(k, int(v)) for k, v in data.items()],
        radius=["40%", "75%"],
        label_opts=opts.LabelOpts(is_show=True, formatter='{b}：{d}%'))
pie.set_global_opts(title_opts=opts.TitleOpts(title="基金经理\n\n从业经验分布", pos_left='center', pos_top='center',
                                               title_textstyle_opts=opts.TextStyleOpts(
                                                   color='#DC143C', font_size=20, font_weight='bold'),
                                               ),
)
pie.render_notebook()

##  管理资产规模

In [16]:
def parase_scale(str_):
    if str(str_).endswith('亿元'):
        return float(str_.replace('亿元', ''))
    else:
        return 0
 
df_2['现任基金资产总规模'] = df_2['现任基金资产总规模'].apply(parase_scale)
df_2['现任基金资产总规模(离散)'] = pd.cut(df_2['现任基金资产总规模'], bins=[0, 5, 20, 50, 100, 300, float('inf')],
                                      labels=["5亿以下", "5-20亿", "20-50亿", '50-100亿', '100-300亿', '300亿以上'])
data = dict(df_2['现任基金资产总规模(离散)'].value_counts())
 
 
pie = Pie(init_opts=opts.InitOpts(theme='light',
                                  width='1000px',
                                  height='600px'))
pie.add("",
        [(k, int(v)) for k, v in data.items()],
        radius=["40%", "75%"],
        label_opts=opts.LabelOpts(is_show=True, formatter='{b}：{d}%'))
pie.set_global_opts(title_opts=opts.TitleOpts(title="基金经理\n\n管理资产规模分布", pos_left='center', pos_top='center',
                                               title_textstyle_opts=opts.TextStyleOpts(
                                                   color='#DC143C', font_size=20, font_weight='bold'),
                                               ),
)
pie.render_notebook()

In [17]:
##  最佳回报率